In [11]:
import pathlib
import os
import warnings
warnings.filterwarnings('ignore')

# Change to the project root directory
project_root = pathlib.Path("/Users/victormp/Desktop/ml/ml-project")
os.chdir(project_root)

import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from darts.models import ARIMA
from darts import TimeSeries
import torch
import torch.nn as nn

# Import Data

In [3]:
data_path = pathlib.Path("data/DCOILWTICO.csv")
print(f"Loading from: {data_path.absolute()}")
print(f"File exists: {data_path.exists()}")

data = pd.read_csv(data_path)
data.rename(columns = {"observation_date" : "date", "DCOILWTICO" : "price"}, inplace  = True)
data["date"] = pd.to_datetime(data["date"])
data = data.set_index("date")
data["return"] = np.log(data["price"]) - np.log(data["price"].shift(1))
returns = data["return"].replace([np.inf, -np.inf], np.nan).dropna().astype("float32")

Loading from: /Users/victormp/Desktop/ml/ml-project/data/DCOILWTICO.csv
File exists: True


# Split Data

In [ ]:
split_point = int(len(returns) * 0.8)  # 80% train, 20% test
y_train = returns.iloc[:split_point]
y_test = returns.iloc[split_point:]

# Convert y_train["return"] and y_test["return"] to darts TimeSeries objects
train_series = TimeSeries.from_values(y_train)
test_series = TimeSeries.from_values(y_test)

# Model Code

In [5]:
from torch.nn.utils import weight_norm


class Classic_TCN(nn.Module):
    """
    Classic TCN (Bai et al., 2028)

    """

    def __init__(self, kernel_size=3, num_filters=64, num_layers=3, dilation_base=2):
        super().__init__()

        # TCN for nonlinear dynamics
        layers = []
        for i in range(num_layers):
            dilation = dilation_base ** i
            padding = (kernel_size - 1) * dilation
            in_channels = 1 if i == 0 else num_filters

            conv = weight_norm(nn.Conv1d(
                in_channels=in_channels,
                out_channels=num_filters,
                kernel_size=kernel_size,
                dilation=dilation,
                padding=padding
            ))
            layers.append(conv)
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.1))

        layers.append(nn.Conv1d(num_filters, 1, kernel_size=1))
        self.tcn = nn.Sequential(*layers)

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(0).unsqueeze(2)
        elif x.dim() == 2:
            x = x.unsqueeze(2)

        batch_size, seq_len, _ = x.shape

        # TCN processes raw time series directly
        x_transposed = x[:, :-1, :].transpose(1, 2)  # Input: all but last timestep
        tcn_output = self.tcn(x_transposed)
        predictions = tcn_output.transpose(1, 2)
        
        targets = x[:, 1:, :]  # Targets: all but first timestep

        # Align lengths (similar to hybrid model)
        target_len = targets.shape[1]
        pred_len = predictions.shape[1]
        if pred_len > target_len:
            predictions = predictions[:, :target_len, :]
        elif pred_len < target_len:
            padding = target_len - pred_len
            predictions = torch.cat([
                torch.zeros(batch_size, padding, 1, device=predictions.device),
                predictions
            ], dim=1)
        
        return predictions, targets

    def predict(self, x, steps=1):
        """
        Multi-step ahead prediction using the Classic TCN model.
        
        Args:
            x: Input sequence tensor of shape (seq_len,) or (batch_size, seq_len) or (batch_size, seq_len, 1)
            steps: Number of steps to predict ahead
        
        Returns:
            predictions: Tensor of shape (steps,) for single sequence or (batch_size, steps) for batch
        
        The model recursively predicts:
            $\hat{y}_{t+h} = \text{TCN}(y_{t-k:t+h-1})$
        """
        self.eval()
        with torch.no_grad():
            # Normalize input dimensions
            if x.dim() == 1:
                x = x.unsqueeze(0).unsqueeze(2)
                single_sequence = True
            elif x.dim() == 2:
                x = x.unsqueeze(2)
                single_sequence = False
            else:
                single_sequence = False
            
            batch_size = x.shape[0]
            
            # Initialize sequence with input
            sequence = x.clone()
            predictions = []
            
            for step in range(steps):
                # TCN prediction on entire sequence so far
                x_transposed = sequence.transpose(1, 2)
                tcn_output = self.tcn(x_transposed)
                
                # Take the last prediction
                next_pred = tcn_output[:, :, -1:].transpose(1, 2)  # Shape: (batch_size, 1, 1)
                
                predictions.append(next_pred)
                
                # Append prediction to sequence for next iteration
                sequence = torch.cat([sequence, next_pred], dim=1)
            
            # Stack predictions
            predictions = torch.cat(predictions, dim=1)
            
            if single_sequence:
                return predictions.squeeze()
            else:
                return predictions.squeeze(2)


#Flexible AR and TCN Additive Hybrid Model
class AdditiveHybrid_AR_TCN(nn.Module):
    """
    Additive Hybrid Model with flexible AR order: y_t = L_t + N_t (following Wang et al. 2013)

    1. AR(p) predicts linear component: L_hat_t = φ_1*y_{t-1} + ... + φ_p*y_{t-p} + c
    2. Additive residuals: e_t = y_t - L_hat_t
    3. TCN models nonlinear pattern: N_hat_t = TCN(e_{t-k:t-1})
    4. Final prediction: y_hat_t = L_hat_t + N_hat_t
    """

    def __init__(self, ar_order=1, kernel_size=3, num_filters=64, num_layers=3, dilation_base=2):
        super().__init__()

        self.ar_order = ar_order

        # AR(p) component - multiple weights for different lags
        self.ar_weights = nn.Parameter(torch.zeros(ar_order))
        self.ar_bias = nn.Parameter(torch.tensor([0.0]))

        # TCN for nonlinear residuals
        layers = []
        for i in range(num_layers):
            dilation = dilation_base ** i
            padding = (kernel_size - 1) * dilation
            in_channels = 1 if i == 0 else num_filters

            conv = weight_norm(nn.Conv1d(
                in_channels=in_channels,
                out_channels=num_filters,
                kernel_size=kernel_size,
                dilation=dilation,
                padding=padding
            ))
            layers.append(conv)
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.1))

        layers.append(nn.Conv1d(num_filters, 1, kernel_size=1))
        self.tcn = nn.Sequential(*layers)

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(0).unsqueeze(2)
        elif x.dim() == 2:
            x = x.unsqueeze(2)

        batch_size, seq_len, _ = x.shape

        # Step 1: AR(p) predictions (linear component)
        # Need at least ar_order + 1 timesteps
        if seq_len <= self.ar_order:
            raise ValueError(f"Sequence length {seq_len} must be > AR order {self.ar_order}")

        # Compute AR predictions: sum of weighted past values
        ar_predictions = torch.zeros(batch_size, seq_len - self.ar_order, 1, device=x.device)
        for i in range(self.ar_order):
            # ar_weights[i] applies to lag (i+1)
            ar_predictions += self.ar_weights[i] * x[:, self.ar_order-1-i:seq_len-1-i, :]
        ar_predictions += self.ar_bias

        # Step 2: ADDITIVE residuals: e_t = y_t - L_hat_t
        targets = x[:, self.ar_order:, :]
        additive_residuals = targets - ar_predictions

        # Step 3: TCN models residuals
        residuals_transposed = additive_residuals.transpose(1, 2)
        tcn_output = self.tcn(residuals_transposed)
        tcn_predictions = tcn_output.transpose(1, 2)

        # Align lengths
        residual_len = ar_predictions.shape[1]
        tcn_len = tcn_predictions.shape[1]
        if tcn_len > residual_len:
            tcn_predictions = tcn_predictions[:, :residual_len, :]
        elif tcn_len < residual_len:
            padding = residual_len - tcn_len
            tcn_predictions = torch.cat([
                torch.zeros(batch_size, padding, 1, device=tcn_predictions.device),
                tcn_predictions
            ], dim=1)

        # Step 4: ADDITIVE combination: y_hat = L_hat + N_hat
        final_predictions = ar_predictions + tcn_predictions

        return final_predictions, targets

    def predict(self, x, steps=1):
        """
        Multi-step ahead prediction using the additive hybrid model.
        
        Args:
            x: Input sequence tensor of shape (seq_len,) or (batch_size, seq_len) or (batch_size, seq_len, 1)
            steps: Number of steps to predict ahead
        
        Returns:
            predictions: Tensor of shape (steps,) for single sequence or (batch_size, steps) for batch
        
        The model recursively predicts:
            $\hat{y}_{t+h} = \hat{L}_{t+h} + \hat{N}_{t+h}$
        
        where:
            $\hat{L}_{t+h} = \sum_{i=1}^{p} \phi_i y_{t+h-i} + c$
            $\hat{N}_{t+h} = \text{TCN}(e_{t+h-k:t+h-1})$
        """
        self.eval()
        with torch.no_grad():
            # Normalize input dimensions
            if x.dim() == 1:
                x = x.unsqueeze(0).unsqueeze(2)
                single_sequence = True
            elif x.dim() == 2:
                x = x.unsqueeze(2)
                single_sequence = False
            else:
                single_sequence = False
            
            batch_size = x.shape[0]
            
            # Initialize sequence with input
            sequence = x.clone()
            predictions = []
            
            for step in range(steps):
                seq_len = sequence.shape[1]
                
                # Step 1: Compute AR(p) prediction for next timestep
                ar_pred = self.ar_bias.expand(batch_size, 1)
                for i in range(self.ar_order):
                    lag_idx = seq_len - 1 - i
                    if lag_idx >= 0:
                        ar_pred += self.ar_weights[i] * sequence[:, lag_idx, :]
                
                # Step 2: Compute residuals for TCN
                # Get recent AR predictions and compute residuals
                if seq_len > self.ar_order:
                    recent_ar = torch.zeros(batch_size, seq_len - self.ar_order, 1, device=x.device)
                    for i in range(self.ar_order):
                        recent_ar += self.ar_weights[i] * sequence[:, self.ar_order-1-i:seq_len-1-i, :]
                    recent_ar += self.ar_bias
                    
                    recent_targets = sequence[:, self.ar_order:, :]
                    residuals = recent_targets - recent_ar
                else:
                    # Not enough history for residuals
                    residuals = torch.zeros(batch_size, 0, 1, device=x.device)
                
                # Step 3: TCN prediction on residuals
                if residuals.shape[1] > 0:
                    residuals_transposed = residuals.transpose(1, 2)
                    tcn_output = self.tcn(residuals_transposed)
                    tcn_pred = tcn_output[:, :, -1:].transpose(1, 2)
                else:
                    tcn_pred = torch.zeros(batch_size, 1, 1, device=x.device)
                
                # Step 4: Additive combination
                next_pred = ar_pred + tcn_pred
                predictions.append(next_pred)
                
                # Append prediction to sequence for next iteration
                # next_pred is already (batch_size, 1, 1), so just concatenate directly
                sequence = torch.cat([sequence, next_pred], dim=1)
            
            # Stack predictions
            predictions = torch.cat(predictions, dim=1)
            
            if single_sequence:
                return predictions.squeeze()
            else:
                return predictions.squeeze(2)

#Flexible AR and TCN Multiplicative Hybrid Model
class MultiplicativeHybrid_AR_TCN(nn.Module):
    """
    Multiplicative Hybrid Model with flexible AR order: y_t = L_t × N_t (following Wang et al. 2013)

    1. AR(p) predicts linear component: L_hat_t = φ_1*y_{t-1} + ... + φ_p*y_{t-p} + c
    2. MULTIPLICATIVE residuals: e_t = y_t / L_hat_t  ← KEY DIFFERENCE!
    3. TCN models nonlinear pattern: N_hat_t = TCN(e_{t-k:t-1})
    4. Final prediction: y_hat_t = L_hat_t × N_hat_t

    Note: For financial returns that cross zero, we use safe division.
    """

    def __init__(self, ar_order=1, kernel_size=3, num_filters=64, num_layers=3, dilation_base=2, epsilon=1e-6):
        super().__init__()

        self.ar_order = ar_order
        self.epsilon = epsilon

        # AR(p) component - multiple weights for different lags
        self.ar_weights = nn.Parameter(torch.zeros(ar_order))
        self.ar_bias = nn.Parameter(torch.tensor([0.0]))

        # TCN for multiplicative factor
        layers = []
        for i in range(num_layers):
            dilation = dilation_base ** i
            padding = (kernel_size - 1) * dilation
            in_channels = 1 if i == 0 else num_filters

            conv = weight_norm(nn.Conv1d(
                in_channels=in_channels,
                out_channels=num_filters,
                kernel_size=kernel_size,
                dilation=dilation,
                padding=padding
            ))
            layers.append(conv)
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.1))

        layers.append(nn.Conv1d(num_filters, 1, kernel_size=1))
        self.tcn = nn.Sequential(*layers)

        # Bias to center output around 1 (multiplicative identity)
        self.tcn_bias = nn.Parameter(torch.tensor([1.0]))

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(0).unsqueeze(2)
        elif x.dim() == 2:
            x = x.unsqueeze(2)

        batch_size, seq_len, _ = x.shape

        # Step 1: AR(p) predictions (linear component)
        # Need at least ar_order + 1 timesteps
        if seq_len <= self.ar_order:
            raise ValueError(f"Sequence length {seq_len} must be > AR order {self.ar_order}")

        # Compute AR predictions: sum of weighted past values
        ar_predictions = torch.zeros(batch_size, seq_len - self.ar_order, 1, device=x.device)
        for i in range(self.ar_order):
            # ar_weights[i] applies to lag (i+1)
            ar_predictions += self.ar_weights[i] * x[:, self.ar_order-1-i:seq_len-1-i, :]
        ar_predictions += self.ar_bias

        targets = x[:, self.ar_order:, :]

        # Step 2: MULTIPLICATIVE residuals: e_t = y_t / L_hat_t
        # Safe division to handle near-zero values
        safe_ar = torch.where(
            torch.abs(ar_predictions) < self.epsilon,
            torch.sign(ar_predictions) * self.epsilon + self.epsilon,
            ar_predictions
        )
        multiplicative_residuals = targets / safe_ar

        # Step 3: TCN models the multiplicative factor
        residuals_transposed = multiplicative_residuals.transpose(1, 2)
        tcn_output = self.tcn(residuals_transposed)
        tcn_predictions = tcn_output.transpose(1, 2) + self.tcn_bias

        # Align lengths
        residual_len = ar_predictions.shape[1]
        tcn_len = tcn_predictions.shape[1]
        if tcn_len > residual_len:
            tcn_predictions = tcn_predictions[:, :residual_len, :]
        elif tcn_len < residual_len:
            padding = residual_len - tcn_len
            tcn_predictions = torch.cat([
                torch.ones(batch_size, padding, 1, device=tcn_predictions.device),
                tcn_predictions
            ], dim=1)

        # Step 4: MULTIPLICATIVE combination: y_hat = L_hat × N_hat
        final_predictions = ar_predictions * tcn_predictions

        return final_predictions, targets

# Train Code

In [6]:
def train_tcn_model(y_train, num_epochs=100, lr=0.001, seed=42):

    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    if not isinstance(y_train, torch.Tensor):
        y_train = torch.FloatTensor(y_train)

    model = Classic_TCN(kernel_size=3, num_filters=64, num_layers=5, dilation_base=4)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        predictions, targets = model(y_train)
        loss = criterion(predictions, targets)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Classic TCN - Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

    return model


def train_additive_model(y_train, ar_order=1, num_epochs=100, lr=0.001, seed=42):
    """
    Train additive hybrid model with specified AR order

    Args:
        y_train: training data
        ar_order: AR order (e.g., 1 for AR(1), 5 for AR(5))
        num_epochs: number of training epochs
        lr: learning rate
    """

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    if not isinstance(y_train, torch.Tensor):
        y_train = torch.FloatTensor(y_train)

    model = AdditiveHybrid_AR_TCN(
        ar_order=ar_order,
        kernel_size=3,
        num_filters=64,
        num_layers=3,
        dilation_base=2
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        predictions, targets = model(y_train)
        loss = criterion(predictions, targets)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Additive Hybrid AR({ar_order}) + TCN - Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

    return model


def train_multiplicative_model(y_train, ar_order=1, num_epochs=100, lr=0.001, seed=42):
    """
    Train multiplicative hybrid model with specified AR order

    Args:
        y_train: training data
        ar_order: AR order (e.g., 1 for AR(1), 5 for AR(5))
        num_epochs: number of training epochs
        lr: learning rate
    """

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    if not isinstance(y_train, torch.Tensor):
        y_train = torch.FloatTensor(y_train)

    model = MultiplicativeHybrid_AR_TCN(
        ar_order=ar_order,
        kernel_size=3,
        num_filters=64,
        num_layers=3,
        dilation_base=2
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        predictions, targets = model(y_train)
        loss = criterion(predictions, targets)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Multiplicative Hybrid AR({ar_order})+TCN - Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

    return model



# Cross Validation TCN

In [9]:
lr = [0.1, 0.01, 0.001, 0.0001]

for eta in lr:
    print(f"Training with learning rate: {eta}")
    model_classic_tcn = train_tcn_model(y_train.values, num_epochs=100, lr=eta)

Training with learning rate: 0.1
Classic TCN - Epoch [10/100], Loss: 0.363129
Classic TCN - Epoch [20/100], Loss: 0.196513
Classic TCN - Epoch [30/100], Loss: 0.044486
Classic TCN - Epoch [40/100], Loss: 0.023688
Classic TCN - Epoch [50/100], Loss: 0.028985
Classic TCN - Epoch [60/100], Loss: 0.023112
Classic TCN - Epoch [70/100], Loss: 0.022556
Classic TCN - Epoch [80/100], Loss: 0.022388
Classic TCN - Epoch [90/100], Loss: 0.022759
Classic TCN - Epoch [100/100], Loss: 0.020597
Training with learning rate: 0.01
Classic TCN - Epoch [10/100], Loss: 0.001505
Classic TCN - Epoch [20/100], Loss: 0.000781
Classic TCN - Epoch [30/100], Loss: 0.000643
Classic TCN - Epoch [40/100], Loss: 0.000624
Classic TCN - Epoch [50/100], Loss: 0.000597
Classic TCN - Epoch [60/100], Loss: 0.000614
Classic TCN - Epoch [70/100], Loss: 0.000602
Classic TCN - Epoch [80/100], Loss: 0.000569
Classic TCN - Epoch [90/100], Loss: 0.000568
Classic TCN - Epoch [100/100], Loss: 0.000581
Training with learning rate: 0.

# Cross Validation Hybrid

In [12]:
AR_ORDER = [1, 5]
for ar_order in AR_ORDER:
    print(f"Training with hybrid AR{ar_order}+TCN")
    for eta in lr:
        print(f"Training with learning rate: {eta}")
        model_classic_tcn = train_additive_model(y_train.values, ar_order = ar_order, num_epochs=100, lr=eta)

Training with hybrid AR1+TCN
Training with learning rate: 0.1
Additive Hybrid AR(1) + TCN - Epoch [10/100], Loss: 0.182410
Additive Hybrid AR(1) + TCN - Epoch [20/100], Loss: 0.030718
Additive Hybrid AR(1) + TCN - Epoch [30/100], Loss: 0.034999
Additive Hybrid AR(1) + TCN - Epoch [40/100], Loss: 0.026803
Additive Hybrid AR(1) + TCN - Epoch [50/100], Loss: 0.018379
Additive Hybrid AR(1) + TCN - Epoch [60/100], Loss: 0.016421
Additive Hybrid AR(1) + TCN - Epoch [70/100], Loss: 0.013523
Additive Hybrid AR(1) + TCN - Epoch [80/100], Loss: 0.013012
Additive Hybrid AR(1) + TCN - Epoch [90/100], Loss: 0.011849
Additive Hybrid AR(1) + TCN - Epoch [100/100], Loss: 0.009379
Training with learning rate: 0.01
Additive Hybrid AR(1) + TCN - Epoch [10/100], Loss: 0.001258
Additive Hybrid AR(1) + TCN - Epoch [20/100], Loss: 0.000733
Additive Hybrid AR(1) + TCN - Epoch [30/100], Loss: 0.000616
Additive Hybrid AR(1) + TCN - Epoch [40/100], Loss: 0.000580
Additive Hybrid AR(1) + TCN - Epoch [50/100], Los

In [ ]:
forecast_horizon = [1, 2, 3, 5, 10, 30, 50]
results_mse_df = pd.DataFrame(columns=[f"{time}-step MSE" for time in forecast_horizon])
results_mae_df = pd.DataFrame(columns=[f"{time}-step MAE" for time in forecast_horizon])

for time in forecast_horizon:
    with torch.no_grad():
        prediction = model_classic_tcn.predict(y_train_tensor[:-150], steps = time).numpy()
    y_test_vals = y_test[:time]
    mse = np.mean((y_test_vals - prediction)**2)
    mae = np.mean(np.absolute(y_test_vals - prediction))
    results_mse_df.loc[f"TCN", f"{time}-step MSE"] = float(f"{mse:.6f}")
    results_mae_df.loc[f"TCN", f"{time}-step MAE"] = float(f"{mae:.6f}")
    # Reset variables
    del prediction, y_test_vals, mse, mae
    
for ar in AR_ORDERS:
    model = globals()[f"model_hybrid_ar{ar}"]
    for time in forecast_horizon:
        with torch.no_grad():
            prediction = model.predict(y_train_tensor[:-150], steps = time).numpy()
        y_test_vals = y_test[:time]
        mse = np.mean((y_test_vals - prediction)**2)
        mae = np.mean(np.absolute(y_test_vals - prediction))
        results_mse_df.loc[f"AR({ar})+TCN", f"{time}-step MSE"] = float(f"{mse:.6f}")
        results_mae_df.loc[f"AR({ar})+TCN", f"{time}-step MAE"] = float(f"{mae:.6f}")
        # Reset variables
        del prediction, y_test_vals, mse, mae